Activity

###### Group: Laura Mantilla - William Moreno - Andres Rincón

In [102]:
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import numpy as np
import imageio
import math

#Open the gif
im = Image.open("catgif.gif")

#Calculate the width and height 
width, height = im.size
print(f"Image size: {width} x {height} pixels")

#Give me the format of the file
file_format = im.format
print(f"File format: {file_format}")

#Canal color
color_mode = im.mode
print(f"Canal color: {color_mode}")

Image size: 300 x 382 pixels
File format: GIF
Canal color: P


1. Filters:

In [103]:
def filter(im, scale):
    changed_frames=[] #Tensor for modified frames
    try:
        while True:
            #Read the next image from the GIF
            im.seek(im.tell() + 1)

            #Convert the image to a NumPy tensor
            tensor = np.array(im)

            #Darkening and Lighting filter
            scale_factor = scale  #Scale factor (between 0 and 1 to decrease intensity, greater than 1 to increase it)
            tensor_filtered = np.clip(tensor * scale_factor, 0, 255).astype(np.uint8) #Filter operation on a tensor, limiting the values of the tensor elements to the range (0,255)

            im_filtered = Image.fromarray(tensor_filtered) #Therefore, the line of code creates a PIL image from the NumPy tensor_filtered array
            
            #Save the filtered image
            #im_filtered.save(f"frame_{im.tell()}.png")
            
            #Add the modified frame to the list
            changed_frames.append(im_filtered)

    except EOFError:
        return changed_frames

#Filter dark
changed_frames_dark = filter(im, 0.5)

im = Image.open("catgif.gif")
#Filter light
changed_frames_light = filter(im, 1.5)

#Save changed frames to a new GIF file
imageio.mimsave("catgif_dark.gif", changed_frames_dark)
imageio.mimsave("catgif_light.gif", changed_frames_light)


2. Add:

In [104]:
#Define the size of the circle mask
circle_mask_size = (20, 20)

#Define the position and speed of the circle
circle_pos = np.array([100, 100]) #Initial position  
circle_speed = 15 #Speed in pixels per frame
circle_rotation_speed = 100 #Rotation speed in degrees per frame
circle_rotation = 0  #Initial rotation angle

#Open the GIF file
im = Image.open("catgif.gif")

def add_motion_circle(im, scale, circle_pos, circle_rotation, circle_speed, circle_rotation_speed):
    '''
    Function to add a moving and rotating circle to a GIF.

        args:
            im(PIL.GifImagePlugin.GifImageFile): GIF image object.
            scale (float): scale factor for the size of the circle.
            circle_pos (np.ndarray): initial position of the circle as a numpy array with two elements (x, y).
            circle_rotation (float): initial rotation angle of the circle in degrees.
            circle_speed (float): The scroll speed of the circle in pixels per frame.
            circle_rotation_speed (float): Circle rotation speed in degrees per frame.

        Returns:
            modified_frames(List[PIL.Image.Image]): List of PIL image objects for each modified frame of the GIF.
    '''
    modified_frames = []

    try:
        while True:
            #Read the next image from the GIF
            im.seek(im.tell() + 1)

            #Draw the circle at the corresponding pixels
            circle_pos_int = np.round(circle_pos).astype(int)
            x1, y1 = circle_pos_int - np.array(circle_mask_size) // 2
            x2, y2 = x1 + circle_mask_size[0], y1 + circle_mask_size[1]
            draw = ImageDraw.Draw(im)
            draw.ellipse((x1, y1, x2, y2), fill=255)

            #Update the position and rotation of the circle for the next frame
            circle_pos = circle_pos.astype(np.float64)
            circle_pos += circle_speed * np.array([math.cos(circle_rotation), math.sin(circle_rotation)])
            circle_rotation += math.radians(circle_rotation_speed)

            #Converts the current image, which is in "P" format to "RGB" format
            im_filtered = im.convert("RGB")

            #Add the modified frame to the list
            modified_frames.append(im_filtered)

    except EOFError:
        return modified_frames
    
#Add the circle to the GIF
modified_frames = add_motion_circle(im, 0.9999, circle_pos, circle_rotation, circle_speed, circle_rotation_speed)

#Create a new GIF from the changed frames
imageio.mimsave("catgif_object.gif", modified_frames)


3. Moments:

In [105]:
im = Image.open("catgif.gif")  # Open the input GIF file

# Convert each frame to grayscale and calculate the mean pixel value
mean_values = []  # Create an empty list to store the mean values of each frame
frames = [] 
try:
    while True:
        im.seek(im.tell() + 1)  # Move to the next frame
        grayscale_im = im.convert("L")  # Convert the frame to grayscale
        a = np.array(grayscale_im)  # Convert the image to a numpy array
        mean_value = np.mean(a)  # Calculate the mean pixel value of the frame
        mean_values.append(mean_value)  # Add the mean value to the list
        frames.append(im.copy())  # Add a copy of the current frame to the list #new
except EOFError:
    pass  # End of file

# Sort the mean values and find the indices of the darkest frames
sorted_indices = np.argsort(mean_values)  # Find the indices that would sort the list
darkest_frames = sorted_indices[:4]  # Get the first 10 indices (i.e., the darkest frames)

print("The darkest frames are:", darkest_frames)  # Print the indices of the darkest frames
#new
# Save the darkest frames as separate image files
for i, frame_idx in enumerate(darkest_frames):
    frame = frames[frame_idx]
    frame.save(f"dark_frame_{i}.png")

The darkest frames are: [ 2  3 37  1]
